#

## 1) Setup

In [ ]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.2/180.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing inst

In [ ]:
!pip install llama-index-readers-file
!pip install llama-index-llms-openai
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (1,237 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123595 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
from pathlib import Path
import re,os
import csv
from bs4 import BeautifulSoup
import pandas as pd
import subprocess
from pathlib import Path
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter, SimpleFileNodeParser
from llama_index.readers.file import FlatReader

## 2) Download the data files

In [ ]:
!mkdir -p 'data/'
!wget 'https://arxiv.org/pdf/2403.16971' -O 'data/AIOS.pdf'
!wget 'https://arxiv.org/pdf/2406.04692' -O 'data/Mixture_of_Agents.pdf'

--2024-08-28 07:07:18--  https://arxiv.org/pdf/2403.16971
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.131.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 569103 (556K) [application/pdf]
Saving to: ‘data/AIOS.pdf’

data/AIOS.pdf       100%[===================>] 555.76K  --.-KB/s    in 0.006s  

2024-08-28 07:07:19 (91.1 MB/s) - ‘data/AIOS.pdf’ saved [569103/569103]

--2024-08-28 07:07:19--  https://arxiv.org/pdf/2406.04692
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.131.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1157463 (1.1M) [application/pdf]
Saving to: ‘data/Mixture_of_Agents.pdf’

data/Mixture_of_Age 100%[===================>]   1.10M  --.-KB/s    in 0.01s   

2024-08-28 07:07:19 (108 MB/s) - ‘data/Mixture_of_Agents.pdf’ saved [1157463/1157463]



## 3) Define the PDF processor to extract the Table of content automatically

In [ ]:
class PDFContentProcessor:
    def __init__(self, source_pdf):
        self.source_pdf = source_pdf
        self.base_name = source_pdf.rstrip('.pdf')
        self.file_name = Path(source_pdf).name

    def transform_pdf_to_html(self):
        # Command to convert PDF to HTML using pdftohtml tool
      # First command to generate the main HTML file
        command = f"pdftohtml -s -i -enc UTF-8 '{self.source_pdf}' '{self.base_name}.html'"
       # Second command to generate the HTML file with the table of contents (ToC)
        command1 = f"pdftohtml -s -i -c -hidden -noframes -zoom 1.5 '{self.source_pdf}' '{self.base_name}.html'"


        try:
            subprocess.run(command, check=True, shell=True)
            subprocess.run(command1, check=True, shell=True)
        except subprocess.CalledProcessError as e:
            print(f"Error executing command: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

    def load_and_process_html(self):
        html_path = Path(f"./{self.base_name}.html")
        html_docs = FlatReader().load_data(html_path)
        parser = HTMLNodeParser(tags=["p"])
        nodes = parser.get_nodes_from_documents(html_docs)
        return self.improve_format(nodes[0].text)

    def improve_format(self, texto):
        texto = texto.replace('\xa0', ' ')
        texto = texto.replace('&#160;', ' ')
        texto = texto.replace('\n\n', '\n')
        texto = re.sub(r'(\.\n)', '.\n\n', texto)
        return texto

    def write_in_file(self, contenido):
        with open(f"{self.base_name}.txt", 'w', encoding='utf-8') as fichero:
            fichero.write(contenido)

    def split_text(self):
        txt_path = Path(f"{self.base_name}.txt")
        txt_docs = FlatReader().load_data(txt_path)
        splitter = SentenceSplitter(chunk_size=512, chunk_overlap=20)
        return splitter.get_nodes_from_documents(txt_docs)

    def process_html_toc(self):
          with open(f"{self.base_name}s.html", 'r', encoding='utf-8') as file:
              soup = BeautifulSoup(file, 'html.parser')

          main_ul = soup.find('ul')
          if not main_ul:
              return []

          list_lenght = len(main_ul.find_all('li', recursive=False))

          # Setting to handle HTML structure correctly
          if list_lenght == 1:
              main_ul = main_ul.find('ul')
          else:
              if main_ul and main_ul.li:
                  # Additional check to prevent deletion of a valid item
                  if not main_ul.li.a or main_ul.li.a.text.strip().lower() != 'introduction':
                      main_ul.li.decompose()

          if not main_ul:
              return []

          entries = []
          self.parse_list_items(main_ul, entries)
          return entries

    def parse_list_items(self, ul, entries, level=1):
        items = ul.find_all('li', recursive=False)
        for item in items:
            a_tag = item.find('a')
            if a_tag and 'href' in a_tag.attrs:
                page = a_tag['href'].split('#')[1]
                title = a_tag.text.strip()
                levels = [False] * 3
                if level <= 3:
                    levels[level - 1] = True
                entries.append([int(page), title] + levels)

            nested_ul = item.find('ul')
            if nested_ul:
                self.parse_list_items(nested_ul, entries, level + 1)

    def write_toc_to_csv(self, toc_entries):
        with open(f"{self.base_name}_output.csv", 'w', newline='', encoding='utf-8') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['page', 'title', 'first_level', 'second_level', 'third_level'])
            csvwriter.writerows(toc_entries)

    def read_and_prepare_sections(self):
        df = pd.read_csv(f"{self.base_name}_output.csv")
        sections_info = [{
            'title': row['title'],
            'page': row['page'],
            'level': 1 if row['first_level'] else 2 if row['second_level'] else 3 if row['third_level'] else None
        } for index, row in df.iterrows()]
        return sections_info

    def add_metadata_to_nodes(self, nodes, sections_info):
        last_index_found_global = 0
        for node_index, node in enumerate(nodes):
            text_content = node.text
            node.metadata = {"page": [], "section": [], "file_name": self.file_name}

            found = False
            last_index_found = last_index_found_global

            for info in sections_info[last_index_found:]:
                pattern = re.escape(info['title']) + r'\n'
                match = re.search(pattern, text_content)
                if match:
                    last_index_found_global += 1
                    node.metadata['section'].append(info['title'])
                    node.metadata['page'].append(info['page'])
                    superiors = self.get_superior_sections(info['title'], sections_info)
                    for sup_title, sup_page in superiors:
                        if sup_title not in node.metadata['section']:
                            node.metadata['section'].insert(0, sup_title)
                            node.metadata['page'].insert(0, sup_page)
                    found = True
                    break

            if not found and node_index > 0:
                node.metadata = nodes[node_index - 1].metadata.copy()

    def get_superior_sections(self, section_title, sections_info):
        results = []
        current_entry = next((entry for entry in sections_info if entry['title'] == section_title), None)

        if not current_entry:
            return results

        current_level = current_entry['level']
        if current_level > 1:
            index_of_current = sections_info.index(current_entry)
            for entry in reversed(sections_info[:index_of_current]):
                if entry['level'] == current_level - 1:
                    results.insert(0, (entry['title'], entry['page']))
                    current_level -= 1
                    if current_level == 1:
                        break

        return results


    def cleanup(self):
            # Get the directory where the PDF file is located
            directory = Path(self.source_pdf).parent
            base_name = Path(self.base_name).name


            for file in directory.iterdir():
                #  Check if the file starts with the base name and it is not a PDF file
                if file.is_file():
                    if file.stem.startswith(base_name) and file.suffix != '.pdf':
                        try:
                            os.remove(file)
                            print(f"Deleted: {file}")
                        except Exception as e:
                            print(f"Error deleting {file}: {e}")
                    else:
                        print(f"Skipped (not matching criteria): {file}")
                else:
                    print(f"Skipped (not a file): {file}")

    @classmethod
    def process_multiple_pdfs(cls, pdf_directory):
        all_nodes = []
        for pdf_file in Path(pdf_directory).glob("*.pdf"):
            processor = cls(str(pdf_file))
            processor.transform_pdf_to_html()
            clean_text = processor.load_and_process_html()
            processor.write_in_file(clean_text)
            nodes = processor.split_text()
            toc_entries = processor.process_html_toc()
            processor.write_toc_to_csv(toc_entries)
            sections_info = processor.read_and_prepare_sections()
            processor.add_metadata_to_nodes(nodes, sections_info)
            all_nodes.extend(nodes)
            processor.cleanup()  # Clean additional files

        all_nodes_dict = {n.node_id: n for n in all_nodes}
        return all_nodes_dict, all_nodes


Process one file

In [ ]:

pdf_directory_or_file = "./data/Mixture_of_Agents.pdf"

if os.path.isfile(pdf_directory_or_file):
    processor = PDFContentProcessor(pdf_directory_or_file)
    processor.transform_pdf_to_html()
    clean_text = processor.load_and_process_html()
    processor.write_in_file(clean_text)
    nodes = processor.split_text()
    toc_entries = processor.process_html_toc()
    processor.write_toc_to_csv(toc_entries)
    sections_info = processor.read_and_prepare_sections()
    processor.add_metadata_to_nodes(nodes, sections_info)
    all_nodes_dict = {n.node_id: n for n in nodes}
    processor.cleanup()  # Limpiar archivos adicionales
else:
    all_nodes_dict,nodes  = PDFContentProcessor.process_multiple_pdfs(pdf_directory_or_file)

for node_id, node in all_nodes_dict.items():
    print(node_id, node.metadata)

Deleted: data/Mixture_of_Agents.html
Deleted: data/Mixture_of_Agents_output.csv
Deleted: data/Mixture_of_Agents.txt
Deleted: data/Mixture_of_Agentss.html
Skipped (not matching criteria): data/Mixture_of_Agents.pdf
Deleted: data/Mixture_of_Agents-html.html
Skipped (not matching criteria): data/AIOS.pdf
20272c45-75b4-434d-8eac-fafcb7b608aa {'page': [1], 'section': ['Introduction'], 'file_name': 'Mixture_of_Agents.pdf'}
29427a73-eff2-4399-8b79-f4ec56845b88 {'page': [1], 'section': ['Introduction'], 'file_name': 'Mixture_of_Agents.pdf'}
40d1a29e-38a7-4fcd-b3cb-788d618c3cf0 {'page': [1], 'section': ['Introduction'], 'file_name': 'Mixture_of_Agents.pdf'}
1839f53e-759e-460e-844b-c1e98466fc20 {'page': [3], 'section': ['Mixture-of-Agents Methodology'], 'file_name': 'Mixture_of_Agents.pdf'}
efbacbf8-68ce-496b-a68c-bf3a96ee447e {'page': [3, 3], 'section': ['Mixture-of-Agents Methodology', 'Mixture-of-Agents'], 'file_name': 'Mixture_of_Agents.pdf'}
9c09eca2-90a0-47a3-a8d9-01ce4d7ea0b7 {'page': [3,

Process multiple files

In [ ]:
pdf_directory_or_file = "./data/"

if os.path.isfile(pdf_directory_or_file):
    processor = PDFContentProcessor(pdf_directory_or_file)
    processor.transform_pdf_to_html()
    clean_text = processor.load_and_process_html()
    processor.write_in_file(clean_text)
    nodes = processor.split_text()
    toc_entries = processor.process_html_toc()
    processor.write_toc_to_csv(toc_entries)
    sections_info = processor.read_and_prepare_sections()
    processor.add_metadata_to_nodes(nodes, sections_info)
    all_nodes_dict = {n.node_id: n for n in nodes}
    processor.cleanup()  # Limpiar archivos adicionales
else:
    all_nodes_dict,nodes  = PDFContentProcessor.process_multiple_pdfs(pdf_directory_or_file)

for node_id, node in all_nodes_dict.items():
    print(node_id, node.metadata)

Deleted: data/Mixture_of_Agents.html
Deleted: data/Mixture_of_Agents_output.csv
Deleted: data/Mixture_of_Agents.txt
Deleted: data/Mixture_of_Agentss.html
Skipped (not matching criteria): data/Mixture_of_Agents.pdf
Deleted: data/Mixture_of_Agents-html.html
Skipped (not matching criteria): data/AIOS.pdf
Deleted: data/AIOS_output.csv
Deleted: data/AIOS-html.html
Deleted: data/AIOSs.html
Skipped (not matching criteria): data/Mixture_of_Agents.pdf
Skipped (not matching criteria): data/AIOS.pdf
Deleted: data/AIOS.html
Deleted: data/AIOS.txt
72f253f3-dff0-46ea-8676-d116beaa972c {'page': [1], 'section': ['Introduction'], 'file_name': 'Mixture_of_Agents.pdf'}
75931586-0dee-4525-a38d-7b8209bb596d {'page': [1], 'section': ['Introduction'], 'file_name': 'Mixture_of_Agents.pdf'}
af7a69fe-da88-43ae-a6c5-a42954de08df {'page': [1], 'section': ['Introduction'], 'file_name': 'Mixture_of_Agents.pdf'}
3dfd36a1-384f-4fef-b46f-9eebed575352 {'page': [3], 'section': ['Mixture-of-Agents Methodology'], 'file_na

In [ ]:
len(nodes)

60

## 4) Set up the OpenAI API for the LLM and the embedding model.






In [ ]:
os.environ["OPENAI_API_KEY"] = 'YOUR_API_KEY'

In [ ]:
import logging
import sys
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

logging.basicConfig(filename='app.log',
                    level=logging.DEBUG,
                    force=True, # Resets any previous configuration
                    )

# Using the LlamaDebugHandler to print the trace
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.settings import Settings

from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

llm_gpt35t = OpenAI(temperature=0.0, model="gpt-3.5-turbo", callback_manager=callback_manager)

## 5) Metafilters with embeddings

In [ ]:
nodes[18].metadata

{'page': [9], 'section': ['Conclusion'], 'file_name': 'Mixture_of_Agents.pdf'}

In [ ]:
nodes[48].metadata

{'page': [9], 'section': ['Conclusions'], 'file_name': 'AIOS.pdf'}

### 5.1) Search for similarity in metadata without applying weights


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import openai

class EmbeddingFilter:
    def __init__(self, api_key, model_name="text-embedding-ada-002"):
        self.openai_client = openai.Client(api_key=api_key)
        self.embedding_model = model_name
        self.calculated_section_embeddings = {}
        self.node_embeddings = {}

    def get_embedding(self, text):
        """Obtain the embedding of a text using OpenAI's API."""
        response = self.openai_client.embeddings.create(
            input=[text],
            model=self.embedding_model,
        )
        return response.data[0].embedding

    def process_nodes(self, nodes):
        """Process each node to calculate and store embeddings."""
        for node_index, node in enumerate(nodes, start=1):
            # Extract the section text from the metadata
            section_text = node.metadata.get('section', [''])[0]
           # print(section_text)

            if section_text:
                # If the embedding for this section has already been calculated, reuse it
                if section_text not in self.calculated_section_embeddings:
                    self.calculated_section_embeddings[section_text] = self.get_embedding(section_text)

                # Assign the embedding and the node ID to the node_embeddings dictionary
                self.node_embeddings[node_index] = self.calculated_section_embeddings[section_text]

    def search_similarity(self, query, similarity_threshold=0.75, top_k=5):
        """Search for similarity between the query embedding and node embeddings."""
        # Obtain the embedding of the query
        query_embedding = np.array(self.get_embedding(query)).reshape(1, -1)

        # List to store results
        results = []

        # Calculate similarity with each embedding in node_embeddings
        for node_id, node_embedding in self.node_embeddings.items():
            node_embedding = np.array(node_embedding).reshape(1, -1)
            similarity = cosine_similarity(query_embedding, node_embedding)[0][0]

            # Filter by similarity threshold
            if similarity >= similarity_threshold:
                results.append((node_id, similarity))

        # Sort the results by similarity in descending order
        sorted_results = sorted(results, key=lambda x: x[1], reverse=True)

        # Return the top_k results
        return sorted_results[:top_k]



In [ ]:

# Initialize the class with your OpenAI API key
api_key = 'YOUR_API_KEY'
processor = EmbeddingFilter(api_key)

# Assume `nodes` is a list of node objects with the required structure
# Process nodes to calculate and store embeddings
processor.process_nodes(nodes)

# Define the query and search parameters
query = "Conclusion"
similarity_threshold = 0.8
top_k = 10

# Find the most similar nodes to the query
similarity_results = processor.search_similarity(query, similarity_threshold, top_k)

# Retrieve the complete nodes for the obtained results
resulting_nodes = [nodes[node_id - 1] for node_id, _ in similarity_results]

# Print the results
for node in resulting_nodes:
    print(f"Node ID: {node.id_}")
    print(f"Similarity: {next(sim for node_id, sim in similarity_results if node_id == nodes.index(node) + 1):.4f}")
    print(f"Metadata: {node.metadata}")
    print(f"Text: {node.text[:200]}...")  # Show a portion of the node's text
    print("-" * 50)

# Print all metadata of the resulting nodes
for node in resulting_nodes:
    print(node.metadata)
    print("-" * 50)  # Separator between nodes

Node ID: 4c9858e6-2c99-4142-848a-dfef39ee74f6
Similarity: 1.0000
Metadata: {'page': [9], 'section': ['Conclusion'], 'file_name': 'Mixture_of_Agents.pdf'}
Text: Additionally, FrugalGPT
(Chen et al., 2023b)
proposed reducing the cost of using LLMs
by employing different models in a cascading manner. In order to better leverage the responses of
multiple models,...
--------------------------------------------------
Node ID: 0c44839a-6a5a-4700-8df8-2aece7b42fe6
Similarity: 1.0000
Metadata: {'page': [9], 'section': ['Conclusion'], 'file_name': 'Mixture_of_Agents.pdf'}
Text: In
addition, we provide insights into improving the design of MoA; systematic optimization of MoA
architecture is an interesting direction for future work.

Limitations.

Our proposed method requires ...
--------------------------------------------------
Node ID: 909b0a23-e705-498a-9248-a8f38787f956
Similarity: 1.0000
Metadata: {'page': [9], 'section': ['Conclusion'], 'file_name': 'Mixture_of_Agents.pdf'}
Text: arXiv prep

### 5.2) Search similarity in metadata applying different weights and normalization

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize, minmax_scale
import openai

class EmbeddingProcessor:
    def __init__(self, api_key, model_name="text-embedding-ada-002", weight_word="Conclusion", weight_factor=1.5):
        self.openai_client = openai.Client(api_key=api_key)
        self.embedding_model = model_name
        self.weight_word = weight_word
        self.weight_factor = weight_factor
        self.calculated_section_embeddings = {}
        self.node_embeddings = {}

    def get_embedding(self, text):
        """Obtain the embedding of a text using OpenAI's API."""
        response = self.openai_client.embeddings.create(
            input=[text],
            model=self.embedding_model,
        )
        return response.data[0].embedding

    def get_weighted_embedding(self, section_text):
        """Obtain and normalize the embedding of a text with custom weighting."""
        embedding = self.get_embedding(section_text)

        # Apply custom weights: increase weight if the section contains the weight word
        weight = self.weight_factor if self.weight_word in section_text else 1.0
        weighted_embedding = np.array(embedding) * weight

        # Normalize the weighted embedding
        normalized_embedding = normalize([weighted_embedding], norm='l2')[0]
        return normalized_embedding

    def process_nodes(self, nodes):
        """Process each node to calculate and store embeddings."""
        for node_index, node in enumerate(nodes, start=1):
            section_text = node.metadata.get('section', [''])[0]
           #print(section_text)

            if section_text:
                # If the embedding for this section has already been calculated, reuse it
                if section_text not in self.calculated_section_embeddings:
                    self.calculated_section_embeddings[section_text] = self.get_weighted_embedding(section_text)

                # Assign the embedding and the node ID to the node_embeddings dictionary
                self.node_embeddings[node_index] = self.calculated_section_embeddings[section_text]

    def search_similarity(self, query, nodes, similarity_threshold=0.75, top_k=5):
        """Search for similarity between the query embedding and node embeddings."""
        # Obtain the embedding of the query
        query_embedding = np.array(self.get_weighted_embedding(query)).reshape(1, -1)

        # List to store results
        results = []

        # Calculate similarity with each embedding in node_embeddings
        for node_id, node_embedding in self.node_embeddings.items():
            node_embedding = np.array(node_embedding).reshape(1, -1)
            similarity = cosine_similarity(query_embedding, node_embedding)[0][0]

            # Apply additional scaling if the section contains the weight word
            section_text = nodes[node_id - 1].metadata.get('section', [''])[0]
            if self.weight_word in section_text:
                similarity *= self.weight_factor  # Increase similarity score for sections containing the weight word

            results.append((node_id, similarity))

        # Rescale the similarity scores between 0 and 1
        similarities = [sim for _, sim in results]
        rescaled_similarities = minmax_scale(similarities, feature_range=(0, 1))

        # Assign the rescaled similarities back to the results
        rescaled_results = [(node_id, sim) for (node_id, _), sim in zip(results, rescaled_similarities)]

        # Filter by similarity threshold
        rescaled_results = [(node_id, sim) for node_id, sim in rescaled_results if sim >= similarity_threshold]

        # Sort the results by similarity in descending order
        sorted_results = sorted(rescaled_results, key=lambda x: x[1], reverse=True)

        # Return the top_k results
        return sorted_results[:top_k]


# Example usage

# Initialize the class with your OpenAI API key and the word to weight
api_key = 'YOUR_API_KEY'
processor = EmbeddingProcessor(api_key, weight_word="Conclu", weight_factor=1.5)

# Assume `nodes` is a list of node objects with the required structure
# Process nodes to calculate and store embeddings
processor.process_nodes(nodes)

# Define the query and search parameters
query = "Conclusion"
similarity_threshold = 0.7
top_k = 15

# Find the most similar nodes to the query
similarity_results = processor.search_similarity(query, nodes, similarity_threshold, top_k)

# Retrieve the complete nodes for the obtained results
resulting_nodes = [nodes[node_id - 1] for node_id, _ in similarity_results]

# Print the results
for node in resulting_nodes:
    print(f"Node ID: {node.id_}")
    print(f"Similarity: {next(sim for node_id, sim in similarity_results if node_id == nodes.index(node) + 1):.4f}")
    print(f"Metadata: {node.metadata}")
    print(f"Text: {node.text[:200]}...")  # Show a portion of the node's text
    print("-" * 50)

# Print all metadata of the resulting nodes
for node in resulting_nodes:
    print(node.metadata)
    print("-" * 50)  # Separator between nodes


Node ID: 4c9858e6-2c99-4142-848a-dfef39ee74f6
Similarity: 1.0000
Metadata: {'page': [9], 'section': ['Conclusion'], 'file_name': 'Mixture_of_Agents.pdf'}
Text: Additionally, FrugalGPT
(Chen et al., 2023b)
proposed reducing the cost of using LLMs
by employing different models in a cascading manner. In order to better leverage the responses of
multiple models,...
--------------------------------------------------
Node ID: 0c44839a-6a5a-4700-8df8-2aece7b42fe6
Similarity: 1.0000
Metadata: {'page': [9], 'section': ['Conclusion'], 'file_name': 'Mixture_of_Agents.pdf'}
Text: In
addition, we provide insights into improving the design of MoA; systematic optimization of MoA
architecture is an interesting direction for future work.

Limitations.

Our proposed method requires ...
--------------------------------------------------
Node ID: 909b0a23-e705-498a-9248-a8f38787f956
Similarity: 1.0000
Metadata: {'page': [9], 'section': ['Conclusion'], 'file_name': 'Mixture_of_Agents.pdf'}
Text: arXiv prep

## 6) Querying

### 6.1 Transform vector store index (Llama Index)

In [ ]:
import logging
import sys
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

logging.basicConfig(filename='app.log',
                    level=logging.DEBUG,
                    force=True,
                    )

# Using the LlamaDebugHandler to print the trace
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager
from llama_index.core.schema import TextNode
from llama_index.core import VectorStoreIndex

# Function to convert a node into a TextNode
def convert_to_text_node(node, similarity):
    return TextNode(
        id_=node.id_,
        text=node.text,
        metadata={
            'section': node.metadata.get('section', [''])[0],
            'file_name': node.metadata.get('file_name', [''])[0],
            'similarity_score': round(similarity, 4)  # Adding similarity score as part of metadata
        }
    )

# Get the actual nodes from the node_id in similarity_results
resulting_nodes = [(nodes[node_id - 1], similarity) for node_id, similarity in similarity_results]

# Convert node results to a TextNode list
text_nodes = [convert_to_text_node(node, similarity) for node, similarity in resulting_nodes]


for text_node in text_nodes:
    print(f"Node ID: {text_node.id_}")
    print(f"Similarity: {text_node.metadata['similarity_score']:.4f}")
    print(f"Metadata: {text_node.metadata}")
    print(f"Text: {text_node.text[:200]}...")  # Displays a portion of the node text
    print("-" * 50)


for text_node in text_nodes:
    print(text_node.metadata)
    print("-" * 50)




Node ID: 4c9858e6-2c99-4142-848a-dfef39ee74f6
Similarity: 1.0000
Metadata: {'section': 'Conclusion', 'file_name': 'M', 'similarity_score': 1.0}
Text: Additionally, FrugalGPT
(Chen et al., 2023b)
proposed reducing the cost of using LLMs
by employing different models in a cascading manner. In order to better leverage the responses of
multiple models,...
--------------------------------------------------
Node ID: 0c44839a-6a5a-4700-8df8-2aece7b42fe6
Similarity: 1.0000
Metadata: {'section': 'Conclusion', 'file_name': 'M', 'similarity_score': 1.0}
Text: In
addition, we provide insights into improving the design of MoA; systematic optimization of MoA
architecture is an interesting direction for future work.

Limitations.

Our proposed method requires ...
--------------------------------------------------
Node ID: 909b0a23-e705-498a-9248-a8f38787f956
Similarity: 1.0000
Metadata: {'section': 'Conclusion', 'file_name': 'M', 'similarity_score': 1.0}
Text: arXiv preprint arXiv:2309.13007
, 2023a.

In [ ]:
from llama_index.core import VectorStoreIndex

In [ ]:
# Create the index using the TextNode
index = VectorStoreIndex(text_nodes,callback_manager=callback_manager)

**********
Trace: index_construction
    |_embedding -> 0.710751 seconds
**********


In [ ]:
# Now you can use the index for queries
query_engine = index.as_query_engine(similarity_top_k=5)
response = query_engine.query("Which are the conclusions of the AIOS paper?")

print(response)

**********
Trace: query
    |_query -> 1.427979 seconds
      |_retrieve -> 0.124469 seconds
        |_embedding -> 0.121983 seconds
      |_synthesize -> 1.302857 seconds
        |_templating -> 2.5e-05 seconds
        |_llm -> 1.290545 seconds
**********
The AIOS paper concludes by proposing the AIOS architecture to facilitate the development and deployment of LLM-based agents, aiming to create a more cohesive, effective, and efficient AIOS-Agent ecosystem.


In [ ]:
from llama_index.core.response.notebook_utils import display_source_node
retrieval=query_engine.retrieve('Which are the conclusions of the AIOS paper?')
for n in retrieval:
  display_source_node(n, source_length=500)

**********
Trace: query
    |_retrieve -> 0.105555 seconds
      |_embedding -> 0.102968 seconds
**********


**Node ID:** b9d598e8-a3f0-4e42-9705-a64e9a0ba7d3<br>**Similarity:** 0.8298229423740328<br>**Text:** Table 5: Effectiveness of agent scheduling, compared with non-scheduled (sequential) execution.

LLM backbone
Agent
Sequential execution (non-scheduled)
Concurrent execution (scheduled)
Waiting time (s)
Turnaround time (s)
Waiting time (s)
Turnaround time (s)
Gemma-2b-it
Math Agent
0.002
±
0.001
2.71
±
0.53
2.50
±
0.05
4.18
±
0.18
Narrative Agent
2.18
±
0.53
3.18
±
0.64
3.34
±
0.17
4.18
±
0.18
Rec Agent
4.78
±
0.96
7.68
±
1.27
3.46
±
0.20
5.91
±
0.19
Gemma-7b-it
Math Agent
0.002
±
0.001
4.95
...<br>

**Node ID:** 0c44839a-6a5a-4700-8df8-2aece7b42fe6<br>**Similarity:** 0.7963888703283242<br>**Text:** In
addition, we provide insights into improving the design of MoA; systematic optimization of MoA
architecture is an interesting direction for future work.

Limitations.

Our proposed method requires iterative aggregation of model responses, which means
the model cannot decide the first token until the last MoA layer is reached. This potentially results
in a high Time to First Token (TTFT), which can negatively impact user experience. To mitigate
this issue, we can limit the number of MoA lay...<br>

**Node ID:** 4c9858e6-2c99-4142-848a-dfef39ee74f6<br>**Similarity:** 0.7902552316056137<br>**Text:** Additionally, FrugalGPT
(Chen et al., 2023b)
proposed reducing the cost of using LLMs
by employing different models in a cascading manner. In order to better leverage the responses of
multiple models,
Jiang et al. (2023)
trained a G
EN
F
USER
, a model that was trained to generate an
improved response to capitalize on the strengths of multiple candidates.

Huang et al. (2024)
proposed
to fuse the outputs of different models by averaging their output probability distributions.

Another line of...<br>

**Node ID:** 1af5bed6-f59f-4a88-9f93-72e4a4921ccb<br>**Similarity:** 0.788824519852953<br>**Text:** Association for Computational Linguistics.

doi: 10.18653/v1/2023.acl-long.792. URL
https://aclanthology.org/2023.acl-long.

792
.

Kojima, T., Gu, S. S., Reid, M., Matsuo, Y., and Iwasawa, Y. Large language models are zero-shot
reasoners.

Advances in neural information processing systems
, 35:22199–22213, 2022.

Liang, T., He, Z., Jiao, W., Wang, X., Wang, Y., Wang, R., Yang, Y., Tu, Z., and Shi, S. Encour-
aging divergent thinking in large language models through multi-agent debate.

arXiv...<br>

**Node ID:** 84b4fb9e-61ed-4761-be37-054394472eb7<br>**Similarity:** 0.7810925171618581<br>**Text:** Llama: Open and efficient foundation language models.

arXiv
preprint arXiv:2302.13971
, 2023a.

Touvron, H., Martin, L., Stone, K., Albert, P., Almahairi, A., Babaei, Y., Bashlykov, N., Batra, S.,
Bhargava, P., Bhosale, S., et al. Llama 2: Open foundation and fine-tuned chat models.

arXiv
preprint arXiv:2307.09288
, 2023b.

Wang, H., Polo, F. M., Sun, Y., Kundu, S., Xing, E., and Yurochkin, M. Fusing models with
complementary expertise. In
The Twelfth International Conference on Learning Re...<br>

### 6.1 Call ChatGPT directly

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="YOUR_API_KEY")

In [ ]:
def get_completion(user_prompt,system_prompt, model="gpt-3.5-turbo"):
    chat_completion = client.chat.completions.create(
        model=model,

        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return chat_completion.choices[0].message.content

In [ ]:
question='Which are the conclusions of the AIOS paper?'
user_prompt=f'Taking account the context " {text_nodes} ". + {question} '
system_prompt = ('''You are an Augmented Retrieval Generation system, you will be provided with a text
context for you to answer a question.
If the answer is not in the context, it indicates that it cannot be answered with that information.''')
get_completion(user_prompt,system_prompt)

'The conclusions of the AIOS paper include proposing the AIOS architecture, which demonstrates the potential to facilitate the development and deployment of LLM-based agents, fostering a more cohesive, effective, and efficient AIOS-Agent ecosystem.'

Thank you for making it to the end of the tutorial! 🎉 I hope you found everything you needed. Happy coding! 🚀